In [1]:
import numpy as np
import pickle
import sys
np.set_printoptions(threshold=sys.maxsize)

## SIN BANDIZADOR

In [ ]:
IAPS_codes= ['Negative','Neutral','Positive']
methods= ['coh','plv','imcoh']

path_excombatants= ('IAPS crudos/Casos/Medida 1/02_Features/')
path_control= ('IAPS crudos/Controles/02_Features/')



N_valences = len(IAPS_codes)
N_methods = len(methods)

XP=[]
XN=[]

Xp = np.zeros((31,N_valences,30,5))
Xn = np.zeros((19,N_valences,30,5))

Xp_cohe = np.zeros((31,N_valences,6,5))
Xn_cohe = np.zeros((19,N_valences,6,5))
for method  in range(N_methods):
    Xp = np.zeros((31,N_valences,30,5))
    Xn = np.zeros((19,N_valences,30,5))

    for valence in range(N_valences):  
        
        if method==1 or method==0:
            File_Excombatants = open(path_excombatants+'Casos_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xp[:,valence,:,:] = pickle.load(File_Excombatants)
            print("#...--------",Xp[:,valence,:,:].shape)
            break;
            File_Excombatants.close()
            File_Control = open(path_control+'Controles_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xn[:,valence,:,:] = pickle.load(File_Control) 
            File_Control.close()
            
        elif method==2:
            #print(method,valence)
            File_Excombatants = open(path_excombatants+'Casos_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xp_cohe[:,valence,:,:] = pickle.load(File_Excombatants) 
            File_Excombatants.close()

            File_Control = open(path_control+'Controles_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xn_cohe[:,valence,:,:] = pickle.load(File_Control) 
            File_Control.close()
            
    if method==0 or method==1:
        XP.append(Xp)
        XN.append(Xn)
          
    
XP.append(Xp_cohe)
XN.append(Xn_cohe)


X=[]
for i in range(len(XP)):
    print(XP[i].shape)
    print(i)
    X.append(np.concatenate((XP[i],XN[i]),axis=0))
    
y = np.ones(50)
y[31:] = - 1 
print(y.shape)

# file= open('Excombatants_Data/y.pckl', 'wb')
# pickle.dump(y,file)
# file.close()

# file= open('Excombatants_Data/X.pckl', 'wb')
# pickle.dump(X,file)
# file.close()

# print(XP[1]==XP[0])

## CON BANDIZADOR

In [2]:
def bandizador(X_old):
    
    N_th=5
    N_band=5
    new_x=np.zeros([X_old.shape[0],X_old.shape[1],N_band,N_th,5])
    for sub in range(X_old.shape[0]):
        for valence in range(X_old.shape[1]):
            th_ind=0
            for th in range(0,5):
                for band in range(0,N_band):
                    ind= N_band*(th)+band
                    new_x[sub,valence,band,th]=X_old[sub,valence,ind]
    new_x2=new_x.reshape(X_old.shape[0],X_old.shape[1],N_band,25)
    #print(new_x)
    return new_x
                
               
        

In [3]:
IAPS_codes= ['Negative','Neutral','Positive']
methods= ['coh','plv']

path_excombatants= ('IAPS crudos/Casos/Medida 1/02_Features/')
path_control= ('IAPS crudos/Controles/02_Features/')



N_valences = len(IAPS_codes)
N_methods = len(methods)
N_bands = 5
N_thresholds = 5

XP=[]
XN=[]

Xp = np.zeros((31,N_valences,N_bands*N_thresholds,5))
Xn = np.zeros((19,N_valences,N_bands*N_thresholds,5))

Xp_cohe = np.zeros((31,N_valences,N_bands,5))
Xn_cohe = np.zeros((19,N_valences,N_bands,5))
for method  in range(N_methods):
    Xp = np.zeros((31,N_valences,N_bands*N_thresholds,5))
    Xn = np.zeros((19,N_valences,N_bands*N_thresholds,5))

    for valence in range(N_valences):  
        
        if method==1 or method==0:
            File_Excombatants = open(path_excombatants+'Casos_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xp[:,valence,:,:] = pickle.load(File_Excombatants)
            print(Xp[:,valence,:,:].shape)
            File_Excombatants.close()
            File_Control = open(path_control+'Controles_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xn[:,valence,:,:] = pickle.load(File_Control) 
            File_Control.close()
            R=Xn
        elif method==2:
            #print(method,valence)
            File_Excombatants = open(path_excombatants+'Casos_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xp_cohe[:,valence,:,:] = pickle.load(File_Excombatants) 
            File_Excombatants.close()

            File_Control = open(path_control+'Controles_IAPS_'+methods[method]+'_'+IAPS_codes[valence],'rb')
            Xn_cohe[:,valence,:,:] = pickle.load(File_Control) 
            File_Control.close()
    print(Xp.shape)     
    new_xp = bandizador(Xp)
    new_xn = bandizador(Xn)
    
            
    if method==0 or method==1:
        XP.append(new_xp)
        XN.append(new_xn)
            
# XP.append(Xp_cohe)
# XN.append(Xn_cohe)

print(XP[0].shape)


X=[]
for i in range(len(XP)):

    X.append(np.concatenate((XP[i],XN[i]),axis=0))
    
print(X[0].shape)
    
y = np.ones(50)
y[31:] = - 1 

file= open('Excombatants_Data/y.pckl', 'wb')
pickle.dump(y,file)
file.close()

file= open('Excombatants_Data/X.pckl', 'wb')
pickle.dump(X,file)
file.close()



(31, 25, 5)
(31, 25, 5)
(31, 25, 5)
(31, 3, 25, 5)
(31, 25, 5)
(31, 25, 5)
(31, 25, 5)
(31, 3, 25, 5)
(31, 3, 5, 5, 5)
(50, 3, 5, 5, 5)


In [ ]:
print(len(XP))